In [1]:
## copy iz started_doing_something_MV_model_ALL_spots_with_CB_cancellation_requested_reports notbuka

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import math
import importlib
from dateutil.relativedelta import relativedelta
import get_started_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

#### Helpers ####
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import db_utils
from helpers import settings

bucket = S3BucketUtils()
################

import calculate_all_vars_for_churn_prediction
import churn_risk_calculation_test
import churn_risk_calculation
import model_testing

import re

import models_to_be_fit

In [3]:
settings.S3_ANALYTICS_BUCKET

'spothopper-analytics-kiza'

In [4]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
hs_list_filename = parameters['hs_list_filename']
hs_list_path = 'churn_analysis/data/'+date_dir+'/'+hs_list_filename

prediction_months = parameters['prediction_months']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'ALL_spots_with_CB_cancellation_requested'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [5]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [6]:
date_dir

'2023_01_01'

In [7]:
website_sessions_wo_contactless = pd.read_csv('../issue_3788_mixpanel_tv/data/' + date_dir + \
                                              '/GA_data/sessions_new.csv')

website_sessions_wo_contactless = website_sessions_wo_contactless.groupby(['spot_id', 'event_time'])\
.sessions.sum().reset_index()

website_sessions_wo_contactless['event_time'] = website_sessions_wo_contactless['event_time']\
.apply(lambda x: x[0:7])

website_sessions_wo_contactless = website_sessions_wo_contactless.groupby(['spot_id', 'event_time'])\
.sessions.sum().reset_index()

website_sessions_wo_contactless = website_sessions_wo_contactless.groupby(['spot_id'])\
.sessions.mean().reset_index()

In [8]:
website_sessions_wo_contactless['sessions'].value_counts()

349.000000     3
49.000000      3
7.000000       3
521.000000     3
13.000000      3
              ..
5416.200000    1
1606.428571    1
1726.000000    1
2680.750000    1
8.500000       1
Name: sessions, Length: 5134, dtype: int64

In [9]:
##### Model 0: all variables - prepare data for the model ####
cols_to_use = read_a_combination_of_variables.\
main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

# data_all_spots = \
# calculate_all_vars_for_churn_prediction.\
# get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis,\
#                               hs_filename=hs_list_filename,\
#                               spots_set=spots_set,\
#                               with_wo_CB=with_wo_CB,\
#                               event_date=event_date, columns=cols,\
#                               data_dir=churn_based_on_behaviour_dir)

#### get data for the model ###
(data_, base_df_, df_timeline_all_vars_) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [10]:
num_of_p_values = 1
num_of_prediction_months = len(prediction_months)

In [11]:
df_res = pd.DataFrame()
models_to_run = [36, 37, 38, 39, 40]
cut_offs = [500, 700, 1000, 1200, 1500, 1700, 2000]
for model_number in models_to_run:
    for ct in cut_offs:
        for e in ('high_end_', 'low_end_'):
            if e=='high_end_':
                spots_ = website_sessions_wo_contactless[website_sessions_wo_contactless['sessions']>=ct]['spot_id'].unique()
            else:
                spots_ = website_sessions_wo_contactless[website_sessions_wo_contactless['sessions']<ct]['spot_id'].unique()
            
            data = data_[data_['spot_id'].isin(spots_)].copy()
            base_df = base_df_[base_df_['spot_id'].isin(spots_)].copy()
            df_timeline_all_vars = df_timeline_all_vars_[df_timeline_all_vars_['spot_id'].isin(spots_)].copy()
            
            cols_to_use = read_a_combination_of_variables.\
            main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')

            model_name = \
            read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                            dir_name='combinations_of_variables_that_are_not_dependent/')
            print("# Model "+ str(model_number) + ": " + model_name)
            
            print('had_'+model_name)

            #### get behavioural variables ####
            (variables_to_use_for_the_model, did_something_last_month_vars, \
             did_something_before_and_didnt_last_month_vars) = \
            get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, \
                                                       variables_to_use_for_the_model=cols_to_use)
            cols = variables_to_use_for_the_model + \
            did_something_before_and_didnt_last_month_vars + did_something_last_month_vars
            
            variable_to_check = [c for c in cols if c.startswith('had_'+model_name) and \
                   re.search('before_and_didnt', c) == None][0]
            
            print(variable_to_check)

            #### data for the model ###
            df_timeline = df_timeline_all_vars.copy()
            vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
            [x for x in cols if x not in base_cols]+\
            [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
            df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

            ### variables to skip ###
            df_timeline.isnull().sum().sum() #OK
            skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
            skip_vars

            ctv = fit_tv_cox_models.fit_the_models_and_print_summaries_kiza(df_timeline=df_timeline, base_df=base_df, \
                                                                 date_of_analysis=date_of_analysis, model_type=model_type, \
                                                                 variables_to_use_for_the_model=cols_to_use,\
                                                                 coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                                 coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                                 additional_higher_p_limit=additional_higher_p_limit, \
                                                                 additional_lower_p_limit=additional_lower_p_limit,\
                                                                 skip_vars=skip_vars, penalizer=penalizer)
            
            df_res = pd.concat([df_res, pd.DataFrame([[model_number, e + str(ct), 
                                                       model_name, variable_to_check, ctv, data.spot_id.nunique()]])])

# Model 36: properly_used_reservations_inquiries
had_properly_used_reservations_inquiries
had_properly_used_reservations_inquiries_last_2_months


### sorted by variable name

## without variables with p >= 0.2

### sorted by variable name

# Model 36: properly_used_reservations_inquiries
had_properly_used_reservations_inquiries
had_properly_used_reservations_inquiries_last_2_months


...

### sorted by variable name

# Model 40: properly_used_online_orders_inquiries
had_properly_used_online_orders_inquiries
had_properly_used_online_orders_inquiries_last_2_months


### sorted by variable name

## without variables with p >= 0.2

### sorted by variable name

In [12]:
df_res.columns = ['model_number', 'category', 'model_name', 'variable_name', 'models', 'spots_num']

In [13]:
df_res.columns.tolist()

['model_number',
 'category',
 'model_name',
 'variable_name',
 'models',
 'spots_num']

In [14]:
def extract_coefficients(variable_name, model):
    model_summary = model.summary
    if variable_name in model_summary.index:
        return(model_summary[model_summary.index==variable_name]['exp(coef)'].iloc[0])
    else:
        return(np.NaN)

def extract_p_values(variable_name, model):
    model_summary = model.summary
    if variable_name in model_summary.index:
        return(model_summary[model_summary.index==variable_name]['p'].iloc[0])
    else:
        return(np.NaN)

In [15]:
df_res['exp(coef)'] = df_res.apply(lambda x: extract_coefficients(variable_name=x['variable_name'], 
                                                                  model=x['models'][0]), axis=1)
df_res['p'] = df_res.apply(lambda x: extract_p_values(variable_name=x['variable_name'], 
                                                                  model=x['models'][0]), axis=1)

df_res['exp(coef).2'] = df_res.apply(lambda x: extract_coefficients(variable_name=x['variable_name'], 
                                                                  model=x['models'][1]), axis=1)
df_res['p.2'] = df_res.apply(lambda x: extract_p_values(variable_name=x['variable_name'], 
                                                                  model=x['models'][1]), axis=1)

In [16]:
# df_res[['model_number', 'category', 'model_name', 'variable_name', 'spots_num', 
#         'exp(coef)', 'p', 'exp(coef).2', 'p.2']]

In [17]:
df_res_export = df_res[['model_number', 'category', 'model_name', 'variable_name', 'spots_num', 
        'exp(coef)', 'p', 'exp(coef).2', 'p.2']]

In [18]:
df_res_export[df_res_export['model_number']==40]

,model_number,category,model_name,variable_name,spots_num,exp(coef),p,exp(coef).2,p.2
0,40,high_end_500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,3719,0.797414,0.212898,0.748947,0.102453
0,40,low_end_500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,658,0.730008,0.501168,0.515060,0.072568
0,40,high_end_700,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,3456,0.825679,0.310714,0.783764,0.184618
0,40,low_end_700,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,921,0.228751,0.001744,0.488618,0.060768
0,40,high_end_1000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,3013,0.788945,0.271955,0.844459,0.423330
0,40,low_end_1000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1364,0.967314,0.905471,0.789561,0.328332
0,40,high_end_1200,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2723,0.865010,0.520446,0.840776,0.417915
0,40,low_end_1200,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1654,0.930838,0.779675,0.830049,0.446582
0,40,high_end_1500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2372,0.903452,0.675025,0.835356,0.433278
0,40,low_end_1500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2005,0.906797,0.681181,0.874214,0.560205


In [19]:
df_res_export[df_res_export['model_number']==40][::-1]

,model_number,category,model_name,variable_name,spots_num,exp(coef),p,exp(coef).2,p.2
0,40,low_end_2000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2530,0.842959,0.414354,0.813978,0.316224
0,40,high_end_2000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1847,0.970845,0.916455,1.048370,0.860352
0,40,low_end_1700,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2222,0.883748,0.580985,0.841872,0.430417
0,40,high_end_1700,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2155,0.872179,0.594640,0.916605,0.721223
0,40,low_end_1500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2005,0.906797,0.681181,0.874214,0.560205
0,40,high_end_1500,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2372,0.903452,0.675025,0.835356,0.433278
0,40,low_end_1200,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1654,0.930838,0.779675,0.830049,0.446582
0,40,high_end_1200,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,2723,0.865010,0.520446,0.840776,0.417915
0,40,low_end_1000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,1364,0.967314,0.905471,0.789561,0.328332
0,40,high_end_1000,properly_used_online_orders_inquiries,had_properly_used_online_orders_inquiries_last...,3013,0.788945,0.271955,0.844459,0.423330


In [20]:
df_res_export[df_res_export['model_number']==36][::-1]\
.to_csv('properly_used_reservations_set_ALL_01_01_2023.csv', index=False)

In [21]:
df_res_export[df_res_export['model_number']==37][::-1]\
.to_csv('properly_used_catering_set_ALL_01_01_2023.csv', index=False)

In [22]:
df_res_export[df_res_export['model_number']==38][::-1]\
.to_csv('properly_used_private_parties_set_ALL_01_01_2023.csv', index=False)

In [23]:
df_res_export[df_res_export['model_number']==39][::-1]\
.to_csv('properly_used_job_listings_set_ALL_01_01_2023.csv', index=False)

In [24]:
df_res_export[df_res_export['model_number']==40][::-1]\
.to_csv('properly_used_online_orders_set_ALL_01_01_2023.csv', index=False)